# Ejercicio: ATP. Comparación de rankings
**Autor**: Miguel Toro, José A. Troyano.      **Revisor**: Fermín Cruz     **Última modificación:** 20/12/2017

En este ejercicio trabajaremos con distintas funciones de manipulación
y comparación de rankings. Los rankings son listas de ítems ordenados por
algún criterio. Hay rankings de muchos tipos: de música, de películas, de
audiencias, deportivos, de resultados de búsqueda, ... 

Usaremos datos de uno de los rankings más conocidos
del ámbito deportivo: el ranking de la ATP (Asociación de Tenistas
Profesionales) que se actualiza semana a semana sobre los resultados del
último año de competición.

A partir de los datos implementaremos una serie de funciones de consulta y
visualización que nos permitirán realizar un análisis de los datos de entrada.
La comparación de rankings la realizaremos  con dos métricas relacionadas con el
test estadístico _Tau de Kendall_:
- Correlación de Kendall: mide el grado de similitud entre dos rankings
- Distancia de Kendall: una métrica, en cierto modo complementaria a la anterior,
  que mide la diferencia entre dos rankings
     
Como siempre, antes de empezar, importaremos los módulos que utilizaremos en el resto del ejercicio:

In [ ]:
import csv
import sys
from collections import defaultdict
from matplotlib import pyplot as plt

## 1. Carga de datos

Dispondremos del TOP-25 del ranking ATP desde la temporada 2008 hasta la 2017.
Por cada temporada tendremos un fichero CSV con los 25 mejores jugadores de
la temporada. Todos los ficheros estarán en la carpeta <code>/csv</code>. En los
ficheros habrá una línea por cada jugador con su nombre y la puntuación de
la temporada. El orden en el que aparecen los jugadores se corresponderá con
su posición en el ranking. Este es el fragmento inicial correspondiente al
ranking de 2008:

    Rafael Nadal,6675
    Roger Federer,5305
    Novak Djokovic,5295
    Andy Murray,3720
    Nikolay Davydenko,2715


La siguiente función será la encargada de leer un fichero correspondiente a una temporada, y construir a partir de él una estructura de datos en memoria:

In [ ]:
def lee_temporada(temporada, ruta='./csv/', prefijo='TOP25-ATP-',
                  extension='.csv'):
    ''' Lee el fichero de una temporada y devuelve un ranking de jugadores

    Toma como entrada el año de la temporada y tres parámetros más que permiten
    componer el nombre completo del fichero:
    - ruta: carpeta en la que se encuentra el fichero
    - prefijo: a anteponer a la temporada para obtener el nombre del fichero
    - extension: por defecto es '.csv'
    En los ficheros de entrada hay una línea para cada jugador con sus dos
    informaciones (nombre y puntos). Usaremos el módulo csv de la librería
    estándar de Python para leer los ficheros de entrada.
    También hay que realizar un pequeño procesamiento con los puntos. Hay que
    pasar del formato cadena (que es lo que se interpreta al leer el csv) a un
    valor numérico (para poder aplicar operaciones matemáticas si fuese
    necesario).
    La salida será una lista de tuplas, una por cada jugador.
    '''
    pass

In [ ]:
# Test de la función lee_temporada
ranking_2008 = lee_temporada(2008)
print(ranking_2008)

Apoyándonos en la función <code>lee_temporada</code> implementaremos la función <code>lee_temporadas</code> que lee todos los ficheros <code>CSV</code> que encuentre en la carpeta que recibe como parámetro. La salida de esta función será un diccionario en el que las claves son las temporadas y los valores son los correspondientes rankings:

In [ ]:
def lee_temporadas(temporadas, ruta='./csv/', prefijo='TOP25-ATP-',
                   extension='.csv'):
    ''' Lee todas las temporadas y devuelve un diccionario de rankings

    Toma como entrada una secuencias con los años de varias temporadas y los
    mismos parámetros usados en la función lee_temporada para construir los nombres
    de ficheros.
    Produce como salida un diccionario en el que las claves son las temporadas
    y los valores son los rankings correspondientes a dichas temporadas.
    '''
    pass

In [ ]:
# Test de la función lee_temporadas
temporadas = range(2008, 2018)
rankings = lee_temporadas(temporadas)
for anyo in rankings:
    print ("Temporada {}:{}...".format(anyo, rankings[anyo][:3]))

## 2. Funciones de consulta

En esta sección veremos una serie de funciones que nos permitirán filtrar y extraer informaciones de la estructura de datos que estamos manejando para representar los partidos (una lista de tuplas). Utilizaremos estas funciones de _consulta_ en distintos puntos del resto del ejercicio.

Las primera función de este tipo es <code>jugadores</code>. Producirá un conjunto con los nombres de los jugadores que han aparecido al menos una vez en uno de los rankings que recibe como parámetro:

In [ ]:
def jugadores(rankings):
    ''' Jugadores presentes en una lista de rankings

    Toma como entrada un diccionario cuyas claves son temporadas y sus
    valores rankings (lista de tuplas (jugador, puntos)).
    Produce como salida el conjunto de jugadores presentes en alguno de esos rankings.
    '''
    pass

In [ ]:
# Test de la función jugadores
nombres = jugadores(rankings)
print(nombres)

Con la estructura que tenemos ahora para los rankings (listas de tuplas jugador-puntos) es muy cómodo acceder a la información de un ranking por la posición de los jugadores. Sin embargo, en algunas ocasiones necesitaremos acceder al ranking a partir del nombre de un jugador. Para ello nos será muy útil disponer de una estructura en la que el nombre de los jugadores sea el punto de acceso. La función <code>ranking2dict</code> nos proporciona esta estructura al construir un ranking en _formato diccionario_ en el que las claves son nombres de jugadores:

In [ ]:
def ranking2dict(ranking):
    ''' Convierte un ranking (lista de tuplas) en un diccionario

    Toma como entrada un ranking en forma de lista de tuplas (jugador, puntos).
    Produce como salida un diccionario cuyas claves son los nombres de los
    jugadores y los valores tuplas (posición, puntos).
    '''
    pass

In [ ]:
# Test de la función ranking_dict
ranking_dict_2008 = ranking2dict(ranking_2008)
print(ranking_dict_2008)

Si un jugador aparece en un ranking diremos que hay una _entrada de ese jugador_ en el ranking. Con la función <code>entradas_jugadores</code> vamos a construir una estructura en la que se agrupen todas las apariciones de un jugador en una serie de rankings. Esta función nos servirá, más adelante, para generar gráficas que muestren la evolución de los jugadores a lo largo del tiempo: 

In [ ]:
def entradas_jugadores(rankings):
    ''' Entradas de cada jugador en todos los rankings

    Toma como entrada un diccionario cuyas claves son temporadas y sus
    valores rankings (lista de tuplas (jugador, puntos)).
    Produce como salida un diccionario en el que a cada jugador se le asocia
    una diccionario con las siguentes claves y valores:
       - clave: temporada
       - valor: tupla (puesto, puntos) ocupado en la correspondiente temporada
    Podremos inicializar ese 'diccionario de diccionarios' con la función
    'defaultdict' del módulo 'collections' de la librería estándar de Python.
    La instrucción que construye un diccionario de diccionarios vacío es:
    
        entradas = defaultdict(dict)
    '''
    pass

In [ ]:
# Test de la función entradas_jugadores
entradas = entradas_jugadores(rankings)
for jugador in list(entradas.keys())[:3]:
    print ("{}:\n{}\n".format(jugador, entradas[jugador]))

La última función de consulta (<code>posiciones_historicas</code>) agregará los distintos resultados de cada jugador para calcular la posición media en el conjunto de sus apariciones en los diferentes rankings de los que se tienen datos:

In [ ]:
def posiciones_historicas(rankings):
    ''' Media de las entradas de cada jugador en todos los rankings

    Toma como entrada un diccionario cuyas claves son temporadas y sus
    valores rankings (lista de tuplas (jugador, puntos)).
    Produce como salida una lista de tuplas (posición-media, jugador) ordenada
    de menor a mayor por posición.
    '''
    pass

In [ ]:
# Test de la función posiciones_historicas
posiciones = posiciones_historicas(rankings)
print(posiciones)

## 3. Funciones de visualización

La primera función de visualización generará una gráfica en la que se mostrará la evolución de varios jugadores a lo largo del tiempo. Tomará como entrada un diccionario de rankings y una lista de jugadores, y producirá una curva por cada jugador con los puestos que ha ocupado a lo largo de los años:


In [ ]:
def evolucion_jugadores(jugadores, rankings):
    ''' Gráfica con la evolución de los jugadores

    Toma como entrada una lista de nombres de jugadores y un diccionario
    cuyas claves son temporadas y sus valores rankings (lista de tuplas
    (jugador, puntos)).
    Produce como salida una gráfica con una línea por cada jugador con
    los puestos que ha ocupado en las distintas temporadas. En el caso
    de que un jugador no aparezca en el ranking de una temporada se le
    asignará un puesto más del último de los que aparezcan en todos los
    rankings.
    Seguiremos el siguiente procedimiento:
       - Construir una lista colores con diez nombres de colores admitidos
         por matplotlib
       - Calcular la lista de temporadas a partir del diccionario rankings
       - Calcular el tamaño máximo de los rankings recibidos como parámetros
         y guardarlo en la variable tam_ranking
       - Usar la función 'entradas_jugadores' para calcular las entradas de
         todos los jugadores en todos los rankings. Completar las temporadas
         en las que los jugadores no aparecen en el ranking con la tupla
         (tam_ranking+1, 0) para el puesto y los puntos, respectivamente.
       - Iterar sobre la lista de jugadores y colores para trazar una curva de
         un color distinto para cada jugador. Calcularemos para cada jugador
         la lista de puestos que ha ocupado y trazaremos la curva con la
         siguiente instrucción:
                 plt.plot(temporadas, puestos, label=j, color=c)
       - Para generar la leyenda e invertir el eje Y (para que los primeros
         puestos aparezcan en la parte alta de la gráfica) finalizaremos con
         las siguientes instrucciones:
                 ax = plt.gca()
                 ax.invert_yaxis()
                 plt.legend()
                 plt.show()
    '''
    pass

In [ ]:
# Test de la función evolucion_jugadores
nombres = ['Rafael Nadal', 'Roger Federer', 'Novak Djokovic']
evolucion_jugadores(nombres, rankings)

La función <code>evolucion_top_jugadores</code> se apoya en la función anterior e incluye un filtro que permite seleccionar el segmento de jugadores a visualizar. El filtro se basa en el orden producido por la función <code>posiciones_historicas</code>. Es posible, por ejemplo, comparar los jugadores con mejores resultados históricos, o bien aquellos que ocupan una posición intermedia en el ranking histórico: 

In [ ]:
def evolucion_top_jugadores(rankings, inicio=None, fin=None):
    ''' Gráfica con la evolución de jugadores según puestos

    Toma como entrada un diccionario cuyas claves son temporadas y sus
    valores rankings (lista de tuplas (jugador, puntos)), y dos valores
    inicio y fin que servirán para determinar el rango de jugadores a mostrar
    en la gráfica.
    Los jugadores a mostrar en la gráfica serán aquellos que se encuentran
    entre las posiciones inicio y fin del ranking histórico calculado con la
    fución posiciones_historicas:
      - Si inicio es None se comenzará desde la posición 1
      - Si fin es None se continuará hasta el último jugador
    Hay que tener en cuenta que los parámetros inicio y fin van de 1 a N,
    mientras que los índices de las listas empiezan con 0.
    El procedimiento a seguir será el siguiente:
      - Recalcular los valores apropiados para inicio y fin
      - Llamar a la función posiciones_historicas y calcular la lista de
        nombres de jugadores a mostrar
      - Llamar a la función evolucion_jugadores para generar la gráfica
    '''
    pass

In [ ]:
# Test de la función evolucion_top_jugadores
evolucion_top_jugadores(rankings, fin=6)                 # Los 6 mejores del ranking histórico
evolucion_top_jugadores(rankings, inicio=6, fin=8)       # Del 6 al 8 en el ranking histórico

La función <code>records_puntuación</code> genera un diagrama de barras en el que se muestran, ordenadas de mayor a menor, las mejores entradas en los distintos rankings en función de los puntos obtenidos por los jugadores. Recibe como parámetros un diccionario de rankings y un límite de barras a mostrar:

In [ ]:
def records_puntuacion(rankings, n=10):
    ''' Gráfica con los mejores registros en puntos

    Toma como entrada un diccionario cuyas claves son temporadas y sus
    valores rankings (lista de tuplas (jugador, puntos)).
    Genera una gráfica de barras con los n mejores registros en puntos
    (por defecto n=10).
    Seguiremos el siguiente procedimiento:
       - Construir una lista con las entradas de todos los rankings
       - Construir a partir de la lista anterior una nueva lista de tuplas
         (puntos, jugador), ordenarla de mayor a menor y extraer las primeras
         n tuplas.
       - A partir de esta lista de tuplas obtener dos listas con los puntos
         y los nombres
    Una vez tengamos las listas de puntos y nombres, podremos generar la
    gráfica con las siguientes instrucciones:
            indices = range(len(nombres))
            plt.bar(indices, puntos)
            plt.xticks(indices, nombres, rotation=80, fontsize=8)
            plt.show()
    '''
    pass

In [ ]:
# Test de la función records_puntuacion
records_puntuacion(rankings, n=12)

## 4. Comparación de rankings

En esta sección usaremos una serie de funciones para implementar un procedimiento de comparación de rankings. El núcleo de este procedimiento será una métrica derivada del test de hipótesis no paramétrico de Kendall. 

### 4.1 Representación vectorial de rankings

Lo primero que necesitaremos para poder comparar los rankings es una representación vectorial de los mismos. En esta representación, cada posición de los vectores se asociará a un jugador, y el valor que se registrará en esa posición será el puesto que dicho jugador ocupa en el ranking.

In [ ]:
def calcula_vector_posiciones(nombres, ranking):
    ''' Obtiene un vector de posiciones alineado con una lista de nombres

    Toma como entrada una lista de nombres y un ranking (lista de tuplas
    (jugador, puntos)).
    Produce como salida un vector con tantas posiciones como elementos haya en
    la lista de nombres. La posición correspondiente a cada nombre de ese
    vector contendrá el puesto de ese jugador en el ranking. En caso de que el
    jugador no aparezca en el ranking se reflejará con el valor 'sys.maxsize'. 
    '''
    pass

In [ ]:
# Test de la calcula_vector_posiciones
nombres = ['David Goffin', 'Roger Federer', 'Novak Djokovic', 'Rafael Nadal']
vector_posiciones = calcula_vector_posiciones(nombres, ranking_2008)
print(vector_posiciones)

### 4.2 Cálculo de parejas concordantes y discordantes

Una vez que tenemos una representación vectorial de los rankings ya estamos en disposición de compararlos. Básicamente lo que haremos será contabilizar concordancias y discordancias:
- Diremos que dos rankings _r1_ y _r2_ son concordantes con respecto a dos jugadores _j1_ y _j2_ si los dos jugadores mantienen la misma relación de orden en ambos rankings.
- Diremos que dos rankings _r1_ y _r2_ son discordantes con respecto a dos jugadores _j1_ y _j2_ si los dos jugadores mantienen distintas relaciones de orden en ambos rankings.

Ambas relaciones se implementan, respectivamente, con las funciones <code>concordante</code> y <code>discordante</code>. Estas funciones toman como parámetros los vectores de posición de dos rankings y las posiciones _i_ y _j_ correspondientes a los jugadores que se usan en la comparación.

In [ ]:
def concordante(vpos1, vpos2, i, j):
    ''' Determina si una pareja de índices son concordantes en dos rankings

    Toma como entrada dos vectores de posición (del mismo tamaño) calculados a
    partir de sendos rankings, y dos índices.
    Produce como salida el siguiente valor booleano:
    - True: si i y j ocupan distintas posiciones relativas en ambos rankings
    - False: si i y j ocupan las mismas posiciones relativas en ambos rankings
    '''
    pass

In [ ]:
# Test de la función concordante
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [13, 10, 11, 12]
print(concordante(vector_pos1, vector_pos2, 1, 2)) # Cierto: 11>12 and 10>11
print(concordante(vector_pos1, vector_pos2, 0, 1)) # Falso:  10<11 and 13>10

In [ ]:
def discordante(vpos1, vpos2, i, j):
    ''' Determina si una pareja de índices son discordantes en dos rankings

    Toma como entrada dos vectores de posición (del mismo tamaño) calculados a
    partir de sendos rankings, y dos índices.
    Produce como salida el siguiente valor booleano:
    - True: si i y j ocupan las mismas posiciones relativas en ambos rankings
    - False: si i y j ocupan distintas posiciones relativas en ambos rankings
    '''
    pass

In [ ]:
# Test de la función discordante
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [13, 10, 11, 12]
print(discordante(vector_pos1, vector_pos2, 1, 2)) # Falso: 11>12 and 10>11
print(discordante(vector_pos1, vector_pos2, 0, 1)) # Cierto:  10<11 and 13>10

Las métricas de Kendall se basan en contar el número de pares concordantes y discordantes entre dos rankings. Para poder calcular ambas cuentas implementaremos la función <code>parejas</code> que, generará todas las parejas de índices posibles, y usará las funciones <code>concordante</code> y <code>discordante</code> para evaluar cada una de las dos condiciones.

In [ ]:
def parejas(vpos1, vpos2, condicion):
    ''' Conjunto de índices de dos rankings que cumplen una condición

    Toma como entrada dos vectores de posición (del mismo tamaño) calculados a
    partir de sendos rankings, y una función para evaluar la condición.
    Produce como salida la lista de parejas de índices que cumplen la
    condición. Se evitarán parejas repetidas (por ejemplo (1, 2) y (2, 1) y
    también las compuestas por el mismo índice (como (1, 1)).
    '''
    pass

In [ ]:
# Test de la función parejas (todas concordantes)
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [18, 19, 20, 21]
print(parejas(vector_pos1, vector_pos2, concordante))
print(parejas(vector_pos1, vector_pos2, discordante))

# Test de la función parejas (todas discordantes)
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [9, 8, 7, 6]
print(parejas(vector_pos1, vector_pos2, concordante))
print(parejas(vector_pos1, vector_pos2, discordante))

# Test de la función parejas (algunas concordantes y otras discordantes)
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [13, 10, 11, 12]
print(parejas(vector_pos1, vector_pos2, concordante))
print(parejas(vector_pos1, vector_pos2, discordante))

### 4.3 Similitud y distancia según el test de Kendall

La correlación (o similitud) de Kendall (https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), también conocida como $\tau$ (_tau_) de Kendall, mide el grado de parecido entre dos rankings. Es muy fácil de calcular una vez que tenemos implementada la función que cuenta el número de parejas concordantes y discordantes entre dos rankings. La fórmula es:

$$
\tau = \frac{número \: de \: parejas \: concordantes \; - número \: de \: parejas \: discordantes}{n(n-1)/2}
$$

Donde $n$ es el número de elementos de ambos rankings. La métrica de similitud toma valores entre $-1$ y $1$, siendo $1$ el valor de similitud de dos rankings exactamente iguales, y $-1$ el valor de similitud de dos rankings en los que no hay ninguna pareja concordante.
     


In [ ]:
def similitud_kendall(vpos1, vpos2):
    ''' Calcula la similitud de Kendall de dos rankings

    Toma como entrada dos vectores de posición (del mismo tamaño) calculados a
    partir de sendos rankings.
    Produce como salida la similitud de Kendall calculada a partir de las
    parejas concordantes y discordantes entre ambos rankings.    
    '''
    pass

In [ ]:
# Test de la función similitud_kendall
#    Todas concordantes
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [18, 19, 20, 21]
print(similitud_kendall(vector_pos1, vector_pos2))
#    Todas discordantes
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [9, 8, 7, 6]
print(similitud_kendall(vector_pos1, vector_pos2))
#    Algunas concordantes y otras discordantes
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [13, 10, 11, 12]
print(similitud_kendall(vector_pos1, vector_pos2))

La distancia de Kendall (https://en.wikipedia.org/wiki/Kendall_tau_distance) es una métrica que mide el grado de diferencia entre dos rankings. También es muy fácil de calcular a partir de la función que cuenta el número de parejas discordantes entre dos rankings. La fórmula es:

$$
\tau = \frac{número \: de \: parejas \: discordantes}{n(n-1)/2}
$$

Donde $n$ es el número de elementos de ambos rankings. La métrica de distancia toma valores entre $0$ y $1$, siendo $0$ la distancia entre dos rankings exactamente iguales, y $1$ la distancia entre dos rankings en los que no hay ninguna pareja concordante.
     

In [ ]:
def distancia_kendall(vpos1, vpos2):
    ''' Calcula la distancia de Kendall de dos rankings

    Toma como entrada dos vectores de posición (del mismo tamaño) calculados a
    partir de sendos rankings.
    Produce como salida la distancia de Kendall calculada a partir de las
    parejas discordantes entre ambos rankings.    
    '''
    pass

In [ ]:
# Test de la función distancia_kendall
#    Todas concordantes
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [18, 19, 20, 21]
print(distancia_kendall(vector_pos1, vector_pos2))
#    Todas discordantes
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [9, 8, 7, 6]
print(distancia_kendall(vector_pos1, vector_pos2))
#    Algunas concordantes y otras discordantes
vector_pos1 = [10, 11, 12, 13]
vector_pos2 = [13, 10, 11, 12]
print(distancia_kendall(vector_pos1, vector_pos2))

### 4.4 Evolución de rankings

En esta última función aprovecharemos todo lo implementado en el resto de esta sección para determinar la evolución del ranking ATP a lo largo de los años. La idea es comparar el ranking de cada año con el anterior, y generar una gráfica que muestre lo _volátil_ que ha sido cada año. Se entenderá por _año volátil_ aquel en el que el ranking ha dado un vuelco importante y, por tanto, la distancia Kendall con el ranking del año anterior ha sido mayor:

In [ ]:
def evolucion_rankings(rankings, metrica=distancia_kendall):
    ''' Calcula la evolución de los rankings durante varias temporadas

    Toma como entrada un diccionario cuyas claves son temporadas y sus
    valores rankings (lista de tuplas (jugador, puntos)), y una función para
    calcular una métrica comparativa de dos rankings.
    Produce como salida una gráfica en la que se muestre la métrica de cada
    temporada con respecto a la anterior.
    Seguiremos el siguiente procedimiento:
        - Constuir una lista ordenada de temporadas a partir de las claves del
          diccionario rankings.
        - Calcular los nombres de los jugadores presentes en todos los
          rankings. Estos nombres se usarán para determinar las posiciones de
          los vectores de posiciones.
        - Construir una lista de metricas resultado de aplicar la función
          metrica a todas las parejas de temporadas consecutivas.
    Una vez tengamos las listas de temporadas y metricas, podremos generar la
    gráfica con las siguientes instrucciones:
            plt.plot(metricas)
            plt.xticks(range(len(metricas)), temporadas)
            plt.show()
    '''
    pass

In [ ]:
# Test de la función evolucion_rankings
evolucion_rankings(rankings)